<a href="https://colab.research.google.com/github/Redcoder815/Deep_Learning_PyTorch/blob/main/BidirectionalLSTMRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import torch
import torch.nn as nn
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)
        out, (h_n, c_n) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out
# Example usage
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BiRNN(input_size=32, hidden_size=64, num_layers=1, output_size=1).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())

In [21]:
import torch
from torch.utils.data import Dataset, DataLoader

class SequenceDataset(Dataset):
    def __init__(self, num_samples=2000, seq_len=25, input_size=32):
        self.X = torch.randn(num_samples, seq_len, input_size)
        self.y = torch.randint(0, 2, (num_samples, 1)).float()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [22]:
dataset = SequenceDataset(num_samples=2000, seq_len=25, input_size=32)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [23]:
model.train()
num_epochs = 10

for epoch in range(num_epochs):
    total_loss = 0

    for X_batch, y_batch in loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 43.7195
Epoch 2, Loss: 43.2473
Epoch 3, Loss: 42.8087
Epoch 4, Loss: 42.2582
Epoch 5, Loss: 41.2104
Epoch 6, Loss: 39.7843
Epoch 7, Loss: 37.3104
Epoch 8, Loss: 33.2521
Epoch 9, Loss: 27.7156
Epoch 10, Loss: 21.1606


In [24]:
import torch
import torch.nn as nn

class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)
        self.sigmoid = nn.Sigmoid()

        # Define initial hidden and cell states as learnable parameters
        # The shape is (num_layers * 2 for bidirectional, 1 for batch size, hidden_size)
        # We use unsqueeze(0) to simulate a batch dimension of 1 for initialization
        self.h0_param = nn.Parameter(torch.randn(num_layers * 2, 1, hidden_size))
        self.c0_param = nn.Parameter(torch.randn(num_layers * 2, 1, hidden_size))

    def forward(self, x):
        # Expand h0_param and c0_param to match the current batch size
        # by repeating them across the batch dimension
        h0 = self.h0_param.repeat(1, x.size(0), 1).to(device)
        c0 = self.c0_param.repeat(1, x.size(0), 1).to(device)

        out, (h_n, c_n) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out

In [25]:
# Example usage
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BiRNN(input_size=32, hidden_size=64, num_layers=1, output_size=1).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())